Trước khi run pip, cài đặt các thư viện bằng câu lệnh sau
```
pip install dash
```

In [1]:
# Cài đặt các thư viện cần thiết
import dash
from dash import dcc
from dash import  html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from keras.models import load_model
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

import numpy as np

# Khởi tạo ứng dụng Dash
app = dash.Dash()
server = app.server

In [2]:
# Lấy dữ liệu EURUSD từ MetaTrader5
import MetaTrader5 as mt
import pandas as pd
from datetime import datetime

mt.initialize()

login = 84193298
password = "-8WyMrVz"
server = "MetaQuotes-Demo"

mt.login(login, password, server)

def get_data_FX(symbol, timeframe, rows):
    ohlc_data = pd.DataFrame(mt.copy_rates_from_pos(symbol, timeframe, 0, rows))
    ohlc_data['time'] = ohlc_data.apply(lambda x: datetime.fromtimestamp(x["time"]), axis=1)
    ohlc_data = ohlc_data.rename(columns={'time': 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'real_volume': 'Volume'})
    ohlc_data.set_index("Date")
    return ohlc_data

df_eur = get_data_FX('EURUSD', mt.TIMEFRAME_M1,1000)

df_eur.head()

,Date,Open,High,Low,Close,tick_volume,spread,Volume
0,2024-07-16 22:00:00,1.09021,1.09037,1.09014,1.09036,46,1,0
1,2024-07-16 22:01:00,1.09036,1.09043,1.09028,1.09028,39,1,0
2,2024-07-16 22:02:00,1.09028,1.09029,1.09019,1.09023,35,1,0
3,2024-07-16 22:03:00,1.09024,1.09024,1.09016,1.09020,26,1,0
4,2024-07-16 22:04:00,1.09020,1.09024,1.09018,1.09021,27,1,0


In [3]:
# Sắp xếp theo ngày, lấy cột ngày và giá đóng cửa

df_eur["Date"]=pd.to_datetime(df_eur.Date,format="%Y-%m-%d")
df_eur.index=df_eur['Date'].values

data=df_eur.sort_index(ascending=True,axis=0)
new_data=pd.DataFrame(index=range(0,len(df_eur)),columns=['Date','Close', 'Open', 'High', 'Low'])

for i in range(0,len(data)):
    new_data.loc[i, "Date"]=data['Date'].iloc[i]
    new_data.loc[i, "Close"]=data["Close"].iloc[i]
    new_data.loc[i, "Open"]=data["Open"].iloc[i]
    new_data.loc[i, "High"]=data["High"].iloc[i]
    new_data.loc[i, "Low"]=data["Low"].iloc[i]

new_data.index=new_data['Date'].values
new_data.drop("Date",axis=1,inplace=True)


# Chuẩn hóa dữ liệu
dataset=new_data['Close'].values
dataset=dataset.reshape(-1,1)

train=dataset[0:987,:]
valid=dataset[987:,:]

scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)


In [4]:
# Lấy mẫu từ dữ liệu để đưa ra dự đoán giá cổ phiếu bằng mô hình LSTM, RNN, XGB
lstm_model=load_model("saved_lstm_model.keras")
rnn_model=load_model("saved_rnn_model.keras")
xgb_model=xgb.Booster()
xgb_model.load_model("saved_xgboost_model.json")


train=new_data[:-100].copy()
valid=new_data[-100:].copy()


col_names = ['Close', 'Open', 'High', 'Low']
for col_name in col_names:
    inputs=new_data[len(new_data)-len(valid)-60:][col_name].values
    inputs=inputs.reshape(-1,1)
    inputs=scaler.transform(inputs)

    X_test=[]
    for i in range(60,inputs.shape[0]):
        X_test.append(inputs[i-60:i,0])
    X_test=np.array(X_test)

    X_test_3D=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))


    closing_price_lstm=lstm_model.predict(X_test_3D)
    closing_price_lstm=scaler.inverse_transform(closing_price_lstm)

    closing_price_rnn=rnn_model.predict(X_test_3D)
    closing_price_rnn=scaler.inverse_transform(closing_price_rnn)

    X_test_DMatrix=xgb.DMatrix(X_test)
    closing_price_xgb=xgb_model.predict(X_test_DMatrix).reshape(-1, 1)
    closing_price_xgb=scaler.inverse_transform(closing_price_xgb)

    valid.loc[:, f'Predictions_{col_name}_LSTM'] = closing_price_lstm
    valid.loc[:, f'Predictions_{col_name}_RNN'] = closing_price_rnn
    valid.loc[:, f'Predictions_{col_name}_XGB'] = closing_price_xgb

print(valid)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
                       Close     Open     High      Low  \
2024-07-17 13:00:00  1.09033  1.09043  1.09045  1.09028   
2024-07-17 13:01:00  1.09029  1.09031  1.09034  1.09028   
2024-07-17 13:02:00   1.0903   1.0903  1.09031  1.09025   
2024-07-17 13:03:00   1.0903  1.09033  1.09033  1.09028   
2024-07-17 13:04:00  1.09029   1.0903  1.09031  1.09025   
...                      ...      ...      ...      ...   
2024-07-17 14:35:00   1.0903  1.09036  1.09036   1.0903   
2024-07-17 14:36:00   1.0903   1.0903  1.09031   1.0903   
2024-07-17 14:37:00  1.09025   1.0903   1.0903  1.09025   
2024-07-17 14:38:00  1.09028  1.09025  1.09028  1.09025   
2024-07-17 14:39:00   1.0903  1.09027  1.09031 

In [5]:
app.layout = html.Div([

    html.H1("Forex Price Analysis Dashboard", style={"textAlign": "center"}),

    dcc.Tabs(id="tabs", children=[

        dcc.Tab(label='EURUSD Actual closing price',children=[
			html.Div([
				html.H2("Actual closing price",style={"textAlign": "center"}),
				dcc.Graph(
					id="Actual Data",
					figure={
						"data":[
							go.Scatter(
								x=new_data.index,
								y=new_data["Close"],
								mode='markers'
							)

						],
						"layout":go.Layout(
							title='scatter plot',
							xaxis={'title':'Date'},
							yaxis={'title':'Closing Rate'}
						)
					}

				),
			])


        ]),
        dcc.Tab(label='EURUSD prediction closing price', children=[
            html.Div([
                html.H1("Prediction closing price",
                        style={'textAlign': 'center'}),

                dcc.Dropdown(id='my-dropdown',
                             options=[{'label': 'LSTM', 'value': 'LSTM'},
                                      {'label': 'RNN','value': 'RNN'},
                                      {'label': 'XGBoost', 'value': 'XGB'}],
                             multi=True,value=['LSTM'],
                             style={"display": "block", "margin-left": "auto",
                                    "margin-right": "auto", "width": "60%"}),
                dcc.Graph(id='prediction'),

            ]),
        ])


    ])
])

@app.callback(Output('prediction', 'figure'),
              [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown):
    dropdown = {"LSTM": "LSTM","RNN": "RNN", "XGB": "XGBoost"}
    trace1 = []
    trace2 = []
    
    trace1.append(
        go.Scatter(
            x=new_data.index, 
            y=new_data["Close"], 
            mode='lines', 
            opacity=0.6, 
            name="Actual Closing Price", 
            marker_color='red',
            textposition='bottom center'
        )
    )

    for model in selected_dropdown:
        trace2.append(
            go.Scatter(
                x=valid.index,
				y=valid["Predictions_Close_" + model],
                mode='lines', 
                opacity=0.7,
                name=f'Predictions_{model}',
                textposition='bottom center'
            )
        )

    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]

    figure = {
        'data': data,
        'layout': go.Layout(colorway=
                            ["#FF0000", '#00FF00', '#00CC33', '#009966', '#006699', '#0033CC'],
        height=600,
        title=f"High and Low Prices for {', '.join(str(dropdown[i]) for i in selected_dropdown)} Over Time",
        xaxis={"title":"Date",
            'rangeselector': {
                'buttons': list([
                    {'count': 1, 'label': '1H','step': 'hour','stepmode': 'backward'},
                    {'count': 1, 'label': '1D','step': 'day','stepmode': 'backward'},
                    {'count': 7, 'label': '1W','step': 'day','stepmode': 'backward'},
                    {'count': 1, 'label': '1M','step': 'month','stepmode': 'backward'},
                    {'count': 6, 'label': '6M','step': 'month', 'stepmode': 'backward'},
                    {'step': 'all'}
                ])},
                'rangeslider': {'visible': True}, 'type': 'date'
            },
            yaxis={"title":"Price (USD)"})
    }
    return figure


if __name__=='__main__':
	app.run_server(debug=True)